In [55]:
from mmcv import Config
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO

In [56]:
model_cfg = \
"vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco"

# load config.
cfg = Config.fromfile(os.path.join("save_dir", model_cfg, "configs.py"))

In [57]:
# checkpoint
checkpoint_path = os.path.join(cfg.work_dir, "best_bbox_mAP_50.pth")
#checkpoint_path = os.path.join(cfg.work_dir, "epoch_24.pth")

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=1, dist=False, shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get("test_cfg"))
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2021-05-20 01:24:50,098 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-20 01:24:50,099 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-20 01:24:50,264 - mmdet - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5.conv2.conv_offset.bias, layer3.6.conv2.conv_offset.

Use load_from_local loader


In [58]:
output = single_gpu_test(model, data_loader, show_score_thr=0.001)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 9.6 task/s, elapsed: 87s, ETA:     0s

In [59]:
try:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
except:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[0][j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f"submission_{model_cfg}.csv"), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.13977632 184.42867 358.8622 206.07355 396....,batch_01_vt/0021.jpg
1,0 0.17377792 284.9238 339.59274 307.17773 376....,batch_01_vt/0028.jpg
2,0 0.10953525 71.73079 169.66852 374.59113 378....,batch_01_vt/0031.jpg
3,0 0.08095689 285.3266 152.40588 363.0498 223.9...,batch_01_vt/0032.jpg
4,0 0.20102112 291.18588 0.21539402 315.59183 23...,batch_01_vt/0070.jpg
